In [1]:
%cd ..

C:\Users\Brown Research\Documents\vr-exoskeleton


# import

Check that the exported ONNX model and weights have been exported properly.

In [45]:
import os

import numpy as np
import onnx
import onnxruntime
import torch

from vr_exoskeleton import gaze_modeling

In [46]:
stamp = '1726587404_lstm_vr-ds2-delta-h64_s16'
path_stamp = os.path.join('output', stamp)

In [47]:
path_onnx = os.path.join(path_stamp, 'val_best.onnx')
model_onnx = onnx.load(path_onnx)
onnx.checker.check_model(model_onnx)

In [48]:
# https://github.com/onnx/onnx-docker/blob/master/onnx-ecosystem/inference_demos/resnet50_modelzoo_onnxruntime_inference.ipynb
session = onnxruntime.InferenceSession(path_onnx)
inputs = session.get_inputs()
print([inp.name for inp in inputs])
shapes = [inp.shape for inp in inputs]
shapes

['input', 'h0', 'c0']


[[1, 1, 9], [1, 1, 64], [1, 1, 64]]

In [49]:
x = {
    inputs[0].name: np.array([[[0.046, -0.051, 0.998, 0.032, -0.051, 0.998, 0, 0, 1]]], dtype=np.float32),
    inputs[1].name: np.zeros(inputs[1].shape, dtype=np.float32),
    inputs[2].name: np.zeros(inputs[2].shape, dtype=np.float32),
}
x

{'input': array([[[ 0.046, -0.051,  0.998,  0.032, -0.051,  0.998,  0.   ,  0.   ,
           1.   ]]], dtype=float32),
 'h0': array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       dtype=float32),
 'c0': array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       dtype=float32)}

In [50]:
y_onnx = session.run([], x)
y_onnx

[array([[[-0.00661951,  0.00647806,  0.22395736]]], dtype=float32),
 array([[[ 0.28214827, -0.30408978,  0.07103308,  0.0605324 ,
           0.07243754,  0.02274194,  0.12180427,  0.09621958,
           0.12453493, -0.32164127,  0.03300498,  0.3157115 ,
          -0.29945344,  0.06810333, -0.04092194,  0.08568035,
           0.12185661,  0.02083877, -0.22673914, -0.12396441,
           0.17571536,  0.32476273,  0.2185961 ,  0.08088748,
           0.31141096, -0.15523273,  0.39072794,  0.2999431 ,
          -0.06120691,  0.12097517,  0.05638696,  0.2652217 ,
          -0.1060916 , -0.08794464,  0.0791367 , -0.29475492,
           0.12426026,  0.01968273, -0.17443576,  0.1074459 ,
           0.09905869,  0.11251564, -0.25023362, -0.04954546,
           0.16618916,  0.0163953 ,  0.00244825, -0.35130298,
           0.34308216,  0.28511602,  0.10213096, -0.08076309,
           0.01526525,  0.02226372, -0.13237923,  0.01449596,
          -0.19927077,  0.15095067,  0.23942937, -0.27816576,
  

In [43]:
weights = torch.load(os.path.join(path_stamp, 'val_best.pth'), weights_only=True)
model = gaze_modeling.GazeLSTM(9, [64])
model.load_state_dict(weights)
y = model(*[torch.tensor(a) for a in x.values()])
y = [a.detach().numpy() for a in y]
y

[array([[[-0.00560898,  0.06662751,  0.48892128]]], dtype=float32),
 array([[[-0.08151086, -0.00668897,  0.01659477, -0.04192491,
           0.08786531,  0.02747281,  0.06131611, -0.11329027,
          -0.1405544 , -0.06886122,  0.02409362, -0.07048728,
           0.00369192,  0.11004268, -0.0383971 ,  0.14117606,
          -0.09314879, -0.11525793,  0.02407523, -0.05248694,
          -0.07082793, -0.05920886, -0.13047113, -0.1154762 ,
           0.1264657 ,  0.03752483, -0.04339955,  0.02098732,
          -0.02307992,  0.09405048,  0.0750456 ,  0.10353816,
           0.01974339, -0.02559475, -0.02636508, -0.10609972,
           0.0333731 ,  0.00956159,  0.09917797,  0.05490723,
          -0.03124635, -0.05806206,  0.09710053,  0.07864764,
           0.11812741, -0.10467769, -0.08562802,  0.09803662,
           0.0765564 , -0.10080887, -0.08384477, -0.18505272,
           0.07662403, -0.0219242 ,  0.1006683 , -0.03550651,
          -0.07214425,  0.08486216, -0.10255342,  0.04277418,
  

In [44]:
[np.allclose(a_onnx, a) for a_onnx, a in zip(y_onnx, y)]

[True, True, True]